In [1]:
# Imports
#---------
import sys
import pandas as pd
import numpy as np
import feather
import os
import gc
import datetime as dt
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from pandas.plotting import register_matplotlib_converters
import matplotlib.ticker as ticker
from matplotlib.dates import DateFormatter
import matplotlib as mpl
from datetime import timezone
import datetime as dt

register_matplotlib_converters()

# File locations
#----------------
the_gases = './oxaria/data/raw/1oxaria/json/gap_filling/'
pngs = './oxaria/data/pngs/gap_filling/'


In [2]:
# Load the df 
#-------------
oxaria1_climate = pd.read_feather(the_gases+'oxaria1_climate_gf.ftr').set_index(['tag','rec'])
oxaria1_climate_lt = oxaria1_climate.iloc[:,[0,1,4]]
del oxaria1_climate

In [3]:
# Checking out the data types 
#-----------------------------
print('\n'+'Loading Oxaria1 climate...\n')
oxaria1_climate_lt.info()



Loading Oxaria1 climate...

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 3693839 entries, ('scs-bgx-536', Timestamp('2020-01-04 09:46:18+0000', tz='UTC')) to ('scs-bgx-543', Timestamp('2021-02-28 23:59:41+0000', tz='UTC'))
Data columns (total 3 columns):
 #   Column   Dtype  
---  ------   -----  
 0   val.hmd  float32
 1   val.tmp  float32
 2   name     object 
dtypes: float32(2), object(1)
memory usage: 262.0+ MB


In [4]:
# Some cleaning incase have missed anything
#---------------------------------------------------------------
oxaria1_climate_lt = oxaria1_climate_lt[~oxaria1_climate_lt.index.duplicated(keep='last')]
oxaria1_climate_lt = oxaria1_climate_lt.reset_index()
oxaria1_climate_lt = oxaria1_climate_lt.dropna(axis=0,subset=['tag']).set_index(['tag','rec'])
oxaria1_climate_lt.sort_index()
print('\n Cleaning duplicates & NAs in index (if any)...\n')


 Cleaning duplicates & NAs in index (if any)...



In [5]:
# Get a list of unique tags
#---------------------------
tags = oxaria1_climate_lt.index.get_level_values(0).unique()
print('\n Unique tags loaded...\n \n', tags)



 Unique tags loaded...
 
 Index(['scs-bgx-536', 'scs-bgx-537', 'scs-bgx-538', 'scs-bgx-539',
       'scs-bgx-540', 'scs-bgx-541', 'scs-bgx-542', 'scs-bgx-543'],
      dtype='object', name='tag')


In [6]:
# Define the start dates for stable operation
# ---------------------------------------------
start_dates = ['2020-09-25T00:00:00', '2020-08-01T00:00:00', '2020-06-05T00:00:00', '2020-01-25T00:00:00',
               '2020-05-01T00:00:00', '2020-03-05T00:00:00', '2020-02-06T00:00:00', '2020-12-07T00:00:00']
dates_list = [dt.datetime.fromisoformat(date).replace(
    tzinfo=timezone.utc) for date in start_dates]
dates_dict = dict(zip(tags, dates_list))
print('\n Applying variable start date filters of each sensor...\n \n'+str(dates_dict))


 Applying variable start date filters of each sensor...
 
{'scs-bgx-536': datetime.datetime(2020, 9, 25, 0, 0, tzinfo=datetime.timezone.utc), 'scs-bgx-537': datetime.datetime(2020, 8, 1, 0, 0, tzinfo=datetime.timezone.utc), 'scs-bgx-538': datetime.datetime(2020, 6, 5, 0, 0, tzinfo=datetime.timezone.utc), 'scs-bgx-539': datetime.datetime(2020, 1, 25, 0, 0, tzinfo=datetime.timezone.utc), 'scs-bgx-540': datetime.datetime(2020, 5, 1, 0, 0, tzinfo=datetime.timezone.utc), 'scs-bgx-541': datetime.datetime(2020, 3, 5, 0, 0, tzinfo=datetime.timezone.utc), 'scs-bgx-542': datetime.datetime(2020, 2, 6, 0, 0, tzinfo=datetime.timezone.utc), 'scs-bgx-543': datetime.datetime(2020, 12, 7, 0, 0, tzinfo=datetime.timezone.utc)}


In [9]:
# Select periods of stable operation from the df of all gases sesnor data
# -------------------------------------------------------------------------
print('\n Applying start date filters...\n')

tmp = []

for k, v in dates_dict.items():
    df = oxaria1_climate_lt.query('tag == @k & rec >= @v')
    tmp.append(df)
tmpdf = pd.concat(tmp)
tmpdf.info(null_counts=True)

# Save to feather
# -----------------
print('\n Writing to  '+the_gases+'oxaria1_climate_stable_536_2feb21/.ftr\n')
tmpdf.reset_index().to_feather(the_gases+'oxaria1_climate_stable_536_2feb21/.ftr')
print('All done! \U0001F600')


 Applying start date filters...

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 3113602 entries, ('scs-bgx-536', Timestamp('2020-09-25 00:00:11+0000', tz='UTC')) to ('scs-bgx-543', Timestamp('2021-02-28 23:59:41+0000', tz='UTC'))
Data columns (total 3 columns):
 #   Column   Non-Null Count    Dtype  
---  ------   --------------    -----  
 0   val.hmd  3113602 non-null  float32
 1   val.tmp  3113602 non-null  float32
 2   name     3113602 non-null  object 
dtypes: float32(2), object(1)
memory usage: 166.1+ MB

 Writing to  ./oxaria/data/raw/1oxaria/json/gap_filling/oxaria1_climate_stable_536_2feb21/.ftr

All done! 😀
